# Reading & Filling Missing Values

In [1]:
import pyspark

In [2]:
import pandas as pd

In [3]:
highest_grossers = pd.read_csv('archive/HighestGrossers.csv')

In [4]:
highest_grossers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   YEAR                   27 non-null     int64 
 1   MOVIE                  27 non-null     object
 2   GENRE                  24 non-null     object
 3   MPAA RATING            27 non-null     object
 4   DISTRIBUTOR            27 non-null     object
 5   TOTAL FOR YEAR         27 non-null     object
 6   TOTAL IN 2019 DOLLARS  27 non-null     object
 7   TICKETS SOLD           27 non-null     object
dtypes: int64(1), object(7)
memory usage: 1.8+ KB


Start pyspark session

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

22/01/25 11:30:46 WARN Utils: Your hostname, rich resolves to a loopback address: 127.0.1.1; using 192.168.68.114 instead (on interface wlp0s20f3)
22/01/25 11:30:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/25 11:30:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark

In [8]:
df_pyspark = spark.read.option("header", "true").csv("archive/HighestGrossers.csv", inferSchema=True)

In [9]:
df_pyspark

DataFrame[YEAR: int, MOVIE: string, GENRE: string, MPAA RATING: string, DISTRIBUTOR: string, TOTAL FOR YEAR: string, TOTAL IN 2019 DOLLARS: string, TICKETS SOLD: string]

In [10]:
df_pyspark.show()

+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+
|YEAR|               MOVIE|    GENRE|MPAA RATING|       DISTRIBUTOR|TOTAL FOR YEAR|TOTAL IN 2019 DOLLARS|TICKETS SOLD|
+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+
|1995|      Batman Forever|    Drama|      PG-13|      Warner Bros.|  $184,031,112|         $387,522,978| 4,23,06,002|
|1996|    Independence Day|Adventure|      PG-13|  20th Century Fox|  $306,169,255|         $634,504,608| 6,92,69,062|
|1997|        Men in Black|Adventure|      PG-13|     Sony Pictures|  $250,650,052|         $500,207,943| 5,46,07,854|
|1998|             Titanic|Adventure|      PG-13|Paramount Pictures|  $443,319,081|         $865,842,808| 9,45,24,324|
|1999|Star Wars Ep. I: ...|Adventure|         PG|  20th Century Fox|  $430,443,350|         $776,153,749| 8,47,32,942|
|2000|How the Grinch St...|Adventure|         PG

In [11]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [12]:
df_pyspark.head(3)

[Row(YEAR=1995, MOVIE='Batman Forever', GENRE='Drama', MPAA RATING='PG-13', DISTRIBUTOR='Warner Bros.', TOTAL FOR YEAR='$184,031,112', TOTAL IN 2019 DOLLARS='$387,522,978', TICKETS SOLD='4,23,06,002'),
 Row(YEAR=1996, MOVIE='Independence Day', GENRE='Adventure', MPAA RATING='PG-13', DISTRIBUTOR='20th Century Fox', TOTAL FOR YEAR='$306,169,255', TOTAL IN 2019 DOLLARS='$634,504,608', TICKETS SOLD='6,92,69,062'),
 Row(YEAR=1997, MOVIE='Men in Black', GENRE='Adventure', MPAA RATING='PG-13', DISTRIBUTOR='Sony Pictures', TOTAL FOR YEAR='$250,650,052', TOTAL IN 2019 DOLLARS='$500,207,943', TICKETS SOLD='5,46,07,854')]

In [13]:
df_pyspark.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MOVIE: string (nullable = true)
 |-- GENRE: string (nullable = true)
 |-- MPAA RATING: string (nullable = true)
 |-- DISTRIBUTOR: string (nullable = true)
 |-- TOTAL FOR YEAR: string (nullable = true)
 |-- TOTAL IN 2019 DOLLARS: string (nullable = true)
 |-- TICKETS SOLD: string (nullable = true)



---

In [14]:
df_pyspark['TICKETS SOLD']

Column<'TICKETS SOLD'>

In [15]:
from pyspark.sql.functions import udf, col
import re

commaRep = udf(lambda x: re.sub(',|\$','', x))
df_pyspark=df_pyspark.withColumn('TICKETS SOLD',commaRep('TICKETS SOLD'))
df_pyspark=df_pyspark.withColumn('TOTAL FOR YEAR',commaRep('TOTAL FOR YEAR'))
df_pyspark=df_pyspark.withColumn('TOTAL IN 2019 DOLLARS',commaRep('TOTAL IN 2019 DOLLARS'))
df_pyspark=df_pyspark.withColumn('TICKETS SOLD',col('TICKETS SOLD').cast("Integer"))
df_pyspark=df_pyspark.withColumn('TOTAL FOR YEAR',col('TOTAL FOR YEAR').cast("Integer"))
df_pyspark=df_pyspark.withColumn('TOTAL IN 2019 DOLLARS',col('TOTAL IN 2019 DOLLARS').cast("Integer"))

df_pyspark.show(3)


+----+----------------+---------+-----------+----------------+--------------+---------------------+------------+
|YEAR|           MOVIE|    GENRE|MPAA RATING|     DISTRIBUTOR|TOTAL FOR YEAR|TOTAL IN 2019 DOLLARS|TICKETS SOLD|
+----+----------------+---------+-----------+----------------+--------------+---------------------+------------+
|1995|  Batman Forever|    Drama|      PG-13|    Warner Bros.|     184031112|            387522978|    42306002|
|1996|Independence Day|Adventure|      PG-13|20th Century Fox|     306169255|            634504608|    69269062|
|1997|    Men in Black|Adventure|      PG-13|   Sony Pictures|     250650052|            500207943|    54607854|
+----+----------------+---------+-----------+----------------+--------------+---------------------+------------+
only showing top 3 rows



In [16]:
df_pyspark.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MOVIE: string (nullable = true)
 |-- GENRE: string (nullable = true)
 |-- MPAA RATING: string (nullable = true)
 |-- DISTRIBUTOR: string (nullable = true)
 |-- TOTAL FOR YEAR: integer (nullable = true)
 |-- TOTAL IN 2019 DOLLARS: integer (nullable = true)
 |-- TICKETS SOLD: integer (nullable = true)



In [17]:
df_pyspark.columns

['YEAR',
 'MOVIE',
 'GENRE',
 'MPAA RATING',
 'DISTRIBUTOR',
 'TOTAL FOR YEAR',
 'TOTAL IN 2019 DOLLARS',
 'TICKETS SOLD']

In [18]:
df_pyspark.head(3)

[Row(YEAR=1995, MOVIE='Batman Forever', GENRE='Drama', MPAA RATING='PG-13', DISTRIBUTOR='Warner Bros.', TOTAL FOR YEAR=184031112, TOTAL IN 2019 DOLLARS=387522978, TICKETS SOLD=42306002),
 Row(YEAR=1996, MOVIE='Independence Day', GENRE='Adventure', MPAA RATING='PG-13', DISTRIBUTOR='20th Century Fox', TOTAL FOR YEAR=306169255, TOTAL IN 2019 DOLLARS=634504608, TICKETS SOLD=69269062),
 Row(YEAR=1997, MOVIE='Men in Black', GENRE='Adventure', MPAA RATING='PG-13', DISTRIBUTOR='Sony Pictures', TOTAL FOR YEAR=250650052, TOTAL IN 2019 DOLLARS=500207943, TICKETS SOLD=54607854)]

In [19]:
# Selecting a column
df_pyspark.select("TICKETS SOLD").show()

+------------+
|TICKETS SOLD|
+------------+
|    42306002|
|    69269062|
|    54607854|
|    94524324|
|    84732942|
|    47006948|
|    53074988|
|    69484746|
|    56337374|
|    71050925|
|    59324582|
|    64628368|
|    48914288|
|    73955652|
|    53614916|
|    52598844|
|    48046812|
|    78311295|
|    50306552|
|    40765637|
+------------+
only showing top 20 rows



In [20]:
# Selecting columns
df_pyspark.select(["MOVIE", "TICKETS SOLD"]).show()

+--------------------+------------+
|               MOVIE|TICKETS SOLD|
+--------------------+------------+
|      Batman Forever|    42306002|
|    Independence Day|    69269062|
|        Men in Black|    54607854|
|             Titanic|    94524324|
|Star Wars Ep. I: ...|    84732942|
|How the Grinch St...|    47006948|
|Harry Potter and ...|    53074988|
|          Spider-Man|    69484746|
|        Finding Nemo|    56337374|
|             Shrek 2|    71050925|
|Star Wars Ep. III...|    59324582|
|Pirates of the Ca...|    64628368|
|        Spider-Man 3|    48914288|
|     The Dark Knight|    73955652|
|Transformers: Rev...|    53614916|
|         Toy Story 3|    52598844|
|Harry Potter and ...|    48046812|
|        The Avengers|    78311295|
|          Iron Man 3|    50306552|
|Guardians of the ...|    40765637|
+--------------------+------------+
only showing top 20 rows



In [21]:
# Similar to pandas describe func
df_pyspark.describe().show()

+-------+-----------------+--------------------+------+-----------+----------------+--------------------+---------------------+--------------------+
|summary|             YEAR|               MOVIE| GENRE|MPAA RATING|     DISTRIBUTOR|      TOTAL FOR YEAR|TOTAL IN 2019 DOLLARS|        TICKETS SOLD|
+-------+-----------------+--------------------+------+-----------+----------------+--------------------+---------------------+--------------------+
|  count|               27|                  27|    24|         27|              27|                  27|                   27|                  27|
|   mean|           2008.0|                null|  null|       null|            null| 4.192771630740741E8|  5.537169798888888E8|6.0449451925925925E7|
| stddev|7.937253933193772|                null|  null|       null|            null|1.6314675402082297E8|  1.699856367346755E8|1.8557383982576847E7|
|    min|             1995|   Avengers: Endgame|Action|          G|20th Century Fox|           184031112| 

In [22]:
df_pyspark.show()

+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+
|YEAR|               MOVIE|    GENRE|MPAA RATING|       DISTRIBUTOR|TOTAL FOR YEAR|TOTAL IN 2019 DOLLARS|TICKETS SOLD|
+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+
|1995|      Batman Forever|    Drama|      PG-13|      Warner Bros.|     184031112|            387522978|    42306002|
|1996|    Independence Day|Adventure|      PG-13|  20th Century Fox|     306169255|            634504608|    69269062|
|1997|        Men in Black|Adventure|      PG-13|     Sony Pictures|     250650052|            500207943|    54607854|
|1998|             Titanic|Adventure|      PG-13|Paramount Pictures|     443319081|            865842808|    94524324|
|1999|Star Wars Ep. I: ...|Adventure|         PG|  20th Century Fox|     430443350|            776153749|    84732942|
|2000|How the Grinch St...|Adventure|         PG

In [23]:
# Adding Columns in data frame
df_pyspark = df_pyspark.withColumn('TEST', df_pyspark["TICKETS SOLD"]*2)

In [24]:
df_pyspark.show()

+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+---------+
|YEAR|               MOVIE|    GENRE|MPAA RATING|       DISTRIBUTOR|TOTAL FOR YEAR|TOTAL IN 2019 DOLLARS|TICKETS SOLD|     TEST|
+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+---------+
|1995|      Batman Forever|    Drama|      PG-13|      Warner Bros.|     184031112|            387522978|    42306002| 84612004|
|1996|    Independence Day|Adventure|      PG-13|  20th Century Fox|     306169255|            634504608|    69269062|138538124|
|1997|        Men in Black|Adventure|      PG-13|     Sony Pictures|     250650052|            500207943|    54607854|109215708|
|1998|             Titanic|Adventure|      PG-13|Paramount Pictures|     443319081|            865842808|    94524324|189048648|
|1999|Star Wars Ep. I: ...|Adventure|         PG|  20th Century Fox|     430443350|            77

In [25]:
# Drop column
df_pyspark = df_pyspark.drop("TEST")

In [26]:
df_pyspark.show()

+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+
|YEAR|               MOVIE|    GENRE|MPAA RATING|       DISTRIBUTOR|TOTAL FOR YEAR|TOTAL IN 2019 DOLLARS|TICKETS SOLD|
+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+
|1995|      Batman Forever|    Drama|      PG-13|      Warner Bros.|     184031112|            387522978|    42306002|
|1996|    Independence Day|Adventure|      PG-13|  20th Century Fox|     306169255|            634504608|    69269062|
|1997|        Men in Black|Adventure|      PG-13|     Sony Pictures|     250650052|            500207943|    54607854|
|1998|             Titanic|Adventure|      PG-13|Paramount Pictures|     443319081|            865842808|    94524324|
|1999|Star Wars Ep. I: ...|Adventure|         PG|  20th Century Fox|     430443350|            776153749|    84732942|
|2000|How the Grinch St...|Adventure|         PG

In [27]:
# Rename the columns
df_pyspark.withColumnRenamed('MOVIE', 'MOVIE TITLE').show()

+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+
|YEAR|         MOVIE TITLE|    GENRE|MPAA RATING|       DISTRIBUTOR|TOTAL FOR YEAR|TOTAL IN 2019 DOLLARS|TICKETS SOLD|
+----+--------------------+---------+-----------+------------------+--------------+---------------------+------------+
|1995|      Batman Forever|    Drama|      PG-13|      Warner Bros.|     184031112|            387522978|    42306002|
|1996|    Independence Day|Adventure|      PG-13|  20th Century Fox|     306169255|            634504608|    69269062|
|1997|        Men in Black|Adventure|      PG-13|     Sony Pictures|     250650052|            500207943|    54607854|
|1998|             Titanic|Adventure|      PG-13|Paramount Pictures|     443319081|            865842808|    94524324|
|1999|Star Wars Ep. I: ...|Adventure|         PG|  20th Century Fox|     430443350|            776153749|    84732942|
|2000|How the Grinch St...|Adventure|         PG

## Pyspark Handling Missing Values

In [278]:
missing_val_df= spark.read.option("header", "true").csv("archive/MissingValues.csv", inferSchema=True)

In [279]:
missing_val_df.show()

+----+--------------+----------------+-----------------------------------+--------------------+
|YEAR|  TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+--------------+----------------+-----------------------------------+--------------------+
|2021|  42,37,74,881|  $3,881,777,912|                     $3,881,777,912|               $9.16|
|2020|  22,36,38,958|  $2,048,534,616|                     $2,048,534,616|               $9.16|
|2019|1,22,85,41,629| $11,253,443,955|                    $11,253,444,050|               $9.16|
|2018|1,31,15,36,128| $11,948,096,650|                    $12,013,670,952|               $9.11|
|2017|1,22,56,39,761| $10,993,991,460|                    $11,226,860,216|               $8.97|
|2016|1,30,25,56,378| $11,267,115,924|                    $11,931,416,424|               $8.65|
|2015|1,32,33,56,776| $11,155,900,636|                    $12,121,948,075|               $8.43|
|2014|1,25,74,02,920| $10,272,985,008|  

In [280]:
## drop columns
missing_val_df.drop('AVERAGE TICKET PRICE').show()

+----+--------------+----------------+-----------------------------------+
|YEAR|  TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|
+----+--------------+----------------+-----------------------------------+
|2021|  42,37,74,881|  $3,881,777,912|                     $3,881,777,912|
|2020|  22,36,38,958|  $2,048,534,616|                     $2,048,534,616|
|2019|1,22,85,41,629| $11,253,443,955|                    $11,253,444,050|
|2018|1,31,15,36,128| $11,948,096,650|                    $12,013,670,952|
|2017|1,22,56,39,761| $10,993,991,460|                    $11,226,860,216|
|2016|1,30,25,56,378| $11,267,115,924|                    $11,931,416,424|
|2015|1,32,33,56,776| $11,155,900,636|                    $12,121,948,075|
|2014|1,25,74,02,920| $10,272,985,008|                    $11,517,810,744|
|2013|1,33,91,68,926| $10,887,446,341|                    $12,266,787,382|
|2012|1,38,09,21,942| $10,992,141,616|                    $12,649,244,986|
|2011|1,28,29,15,168| $10

In [281]:
missing_val_df.na.drop().show()

+----+--------------+----------------+-----------------------------------+--------------------+
|YEAR|  TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+--------------+----------------+-----------------------------------+--------------------+
|2021|  42,37,74,881|  $3,881,777,912|                     $3,881,777,912|               $9.16|
|2020|  22,36,38,958|  $2,048,534,616|                     $2,048,534,616|               $9.16|
|2019|1,22,85,41,629| $11,253,443,955|                    $11,253,444,050|               $9.16|
|2018|1,31,15,36,128| $11,948,096,650|                    $12,013,670,952|               $9.11|
|2017|1,22,56,39,761| $10,993,991,460|                    $11,226,860,216|               $8.97|
|2016|1,30,25,56,378| $11,267,115,924|                    $11,931,416,424|               $8.65|
|2015|1,32,33,56,776| $11,155,900,636|                    $12,121,948,075|               $8.43|
|2014|1,25,74,02,920| $10,272,985,008|  

In [282]:
# Attach function to pyspark
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

In [283]:
missing_val_df.shape()

(27, 5)

In [284]:
missing_val_df.count()

27

In [285]:
# by default how is "any". If how is "all", it will look for rows with all values as null. See more details when you shift+tab after open parenthesis
missing_val_df.na.drop(how="all").show()

+----+--------------+----------------+-----------------------------------+--------------------+
|YEAR|  TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+--------------+----------------+-----------------------------------+--------------------+
|2021|  42,37,74,881|  $3,881,777,912|                     $3,881,777,912|               $9.16|
|2020|  22,36,38,958|  $2,048,534,616|                     $2,048,534,616|               $9.16|
|2019|1,22,85,41,629| $11,253,443,955|                    $11,253,444,050|               $9.16|
|2018|1,31,15,36,128| $11,948,096,650|                    $12,013,670,952|               $9.11|
|2017|1,22,56,39,761| $10,993,991,460|                    $11,226,860,216|               $8.97|
|2016|1,30,25,56,378| $11,267,115,924|                    $11,931,416,424|               $8.65|
|2015|1,32,33,56,776| $11,155,900,636|                    $12,121,948,075|               $8.43|
|2014|1,25,74,02,920| $10,272,985,008|  

In [286]:
missing_val_df.na.drop(how="any", thresh=2).show() # At least 2 non-null values should be present

+----+--------------+----------------+-----------------------------------+--------------------+
|YEAR|  TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+--------------+----------------+-----------------------------------+--------------------+
|2021|  42,37,74,881|  $3,881,777,912|                     $3,881,777,912|               $9.16|
|2020|  22,36,38,958|  $2,048,534,616|                     $2,048,534,616|               $9.16|
|2019|1,22,85,41,629| $11,253,443,955|                    $11,253,444,050|               $9.16|
|2018|1,31,15,36,128| $11,948,096,650|                    $12,013,670,952|               $9.11|
|2017|1,22,56,39,761| $10,993,991,460|                    $11,226,860,216|               $8.97|
|2016|1,30,25,56,378| $11,267,115,924|                    $11,931,416,424|               $8.65|
|2015|1,32,33,56,776| $11,155,900,636|                    $12,121,948,075|               $8.43|
|2014|1,25,74,02,920| $10,272,985,008|  

In [287]:
missing_val_df.na.drop(how="any", subset=["TICKETS SOLD"]).show()

+----+--------------+----------------+-----------------------------------+--------------------+
|YEAR|  TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+--------------+----------------+-----------------------------------+--------------------+
|2021|  42,37,74,881|  $3,881,777,912|                     $3,881,777,912|               $9.16|
|2020|  22,36,38,958|  $2,048,534,616|                     $2,048,534,616|               $9.16|
|2019|1,22,85,41,629| $11,253,443,955|                    $11,253,444,050|               $9.16|
|2018|1,31,15,36,128| $11,948,096,650|                    $12,013,670,952|               $9.11|
|2017|1,22,56,39,761| $10,993,991,460|                    $11,226,860,216|               $8.97|
|2016|1,30,25,56,378| $11,267,115,924|                    $11,931,416,424|               $8.65|
|2015|1,32,33,56,776| $11,155,900,636|                    $12,121,948,075|               $8.43|
|2014|1,25,74,02,920| $10,272,985,008|  

In [288]:
missing_val_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- TICKETS SOLD: string (nullable = true)
 |-- TOTAL BOX OFFICE: string (nullable = true)
 |-- TOTAL INFLATION ADJUSTED BOX OFFICE: string (nullable = true)
 |-- AVERAGE TICKET PRICE: string (nullable = true)



In [289]:
# Filling the Missing Value
missing_val_df.na.fill('Missing Values').show() # note that Year wasn't filled because Year is int, while the other columns are str

+----+--------------+----------------+-----------------------------------+--------------------+
|YEAR|  TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+--------------+----------------+-----------------------------------+--------------------+
|2021|  42,37,74,881|  $3,881,777,912|                     $3,881,777,912|               $9.16|
|2020|  22,36,38,958|  $2,048,534,616|                     $2,048,534,616|               $9.16|
|2019|1,22,85,41,629| $11,253,443,955|                    $11,253,444,050|               $9.16|
|2018|1,31,15,36,128| $11,948,096,650|                    $12,013,670,952|               $9.11|
|2017|1,22,56,39,761| $10,993,991,460|                    $11,226,860,216|               $8.97|
|2016|1,30,25,56,378| $11,267,115,924|                    $11,931,416,424|               $8.65|
|2015|1,32,33,56,776| $11,155,900,636|                    $12,121,948,075|               $8.43|
|2014|1,25,74,02,920| $10,272,985,008|  

In [290]:
### Filling the Missing Value
missing_val_df.na.fill(999, ['Year']).show()

+----+--------------+----------------+-----------------------------------+--------------------+
|YEAR|  TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+--------------+----------------+-----------------------------------+--------------------+
|2021|  42,37,74,881|  $3,881,777,912|                     $3,881,777,912|               $9.16|
|2020|  22,36,38,958|  $2,048,534,616|                     $2,048,534,616|               $9.16|
|2019|1,22,85,41,629| $11,253,443,955|                    $11,253,444,050|               $9.16|
|2018|1,31,15,36,128| $11,948,096,650|                    $12,013,670,952|               $9.11|
|2017|1,22,56,39,761| $10,993,991,460|                    $11,226,860,216|               $8.97|
|2016|1,30,25,56,378| $11,267,115,924|                    $11,931,416,424|               $8.65|
|2015|1,32,33,56,776| $11,155,900,636|                    $12,121,948,075|               $8.43|
|2014|1,25,74,02,920| $10,272,985,008|  

In [291]:
missing_val_df

DataFrame[YEAR: int, TICKETS SOLD: string, TOTAL BOX OFFICE: string, TOTAL INFLATION ADJUSTED BOX OFFICE: string, AVERAGE TICKET PRICE: string]

In [292]:
missing_val_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- TICKETS SOLD: string (nullable = true)
 |-- TOTAL BOX OFFICE: string (nullable = true)
 |-- TOTAL INFLATION ADJUSTED BOX OFFICE: string (nullable = true)
 |-- AVERAGE TICKET PRICE: string (nullable = true)



In [293]:
from pyspark.sql.functions import udf, col, when
import re

commaRep = udf(lambda x: re.sub(',|\$| ','', x))
missing_val_df = missing_val_df.na.fill("", ["TICKETS SOLD"]).withColumn('TICKETS SOLD',commaRep('TICKETS SOLD'))
missing_val_df = missing_val_df.withColumn('TICKETS SOLD', when((missing_val_df["TICKETS SOLD"] == ""), None).otherwise(missing_val_df["TICKETS SOLD"]))

# missing_val_df.show(3)


In [294]:
missing_val_df.show()

+----+------------+----------------+-----------------------------------+--------------------+
|YEAR|TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+------------+----------------+-----------------------------------+--------------------+
|2021|   423774881|  $3,881,777,912|                     $3,881,777,912|               $9.16|
|2020|   223638958|  $2,048,534,616|                     $2,048,534,616|               $9.16|
|2019|  1228541629| $11,253,443,955|                    $11,253,444,050|               $9.16|
|2018|  1311536128| $11,948,096,650|                    $12,013,670,952|               $9.11|
|2017|  1225639761| $10,993,991,460|                    $11,226,860,216|               $8.97|
|2016|  1302556378| $11,267,115,924|                    $11,931,416,424|               $8.65|
|2015|  1323356776| $11,155,900,636|                    $12,121,948,075|               $8.43|
|2014|  1257402920| $10,272,985,008|                    $11,

In [295]:
missing_val_df = missing_val_df.na.fill("", ["TOTAL INFLATION ADJUSTED BOX OFFICE"]).withColumn('TOTAL INFLATION ADJUSTED BOX OFFICE',commaRep('TOTAL INFLATION ADJUSTED BOX OFFICE'))


In [296]:
missing_val_df = missing_val_df.withColumn('TOTAL INFLATION ADJUSTED BOX OFFICE',when((missing_val_df["TOTAL INFLATION ADJUSTED BOX OFFICE"] == ""), None).otherwise(missing_val_df["TOTAL INFLATION ADJUSTED BOX OFFICE"]))

In [297]:
missing_val_df.show()

+----+------------+----------------+-----------------------------------+--------------------+
|YEAR|TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+------------+----------------+-----------------------------------+--------------------+
|2021|   423774881|  $3,881,777,912|                         3881777912|               $9.16|
|2020|   223638958|  $2,048,534,616|                         2048534616|               $9.16|
|2019|  1228541629| $11,253,443,955|                        11253444050|               $9.16|
|2018|  1311536128| $11,948,096,650|                        12013670952|               $9.11|
|2017|  1225639761| $10,993,991,460|                        11226860216|               $8.97|
|2016|  1302556378| $11,267,115,924|                        11931416424|               $8.65|
|2015|  1323356776| $11,155,900,636|                        12121948075|               $8.43|
|2014|  1257402920| $10,272,985,008|                        

In [298]:
missing_val_df=missing_val_df.withColumn('TICKETS SOLD',col('TICKETS SOLD').cast("Integer"))

In [299]:
missing_val_df.show()

+----+------------+----------------+-----------------------------------+--------------------+
|YEAR|TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|
+----+------------+----------------+-----------------------------------+--------------------+
|2021|   423774881|  $3,881,777,912|                         3881777912|               $9.16|
|2020|   223638958|  $2,048,534,616|                         2048534616|               $9.16|
|2019|  1228541629| $11,253,443,955|                        11253444050|               $9.16|
|2018|  1311536128| $11,948,096,650|                        12013670952|               $9.11|
|2017|  1225639761| $10,993,991,460|                        11226860216|               $8.97|
|2016|  1302556378| $11,267,115,924|                        11931416424|               $8.65|
|2015|  1323356776| $11,155,900,636|                        12121948075|               $8.43|
|2014|  1257402920| $10,272,985,008|                        

In [300]:
missing_val_df = missing_val_df.withColumn('TOTAL INFLATION ADJUSTED BOX OFFICE',col('TOTAL INFLATION ADJUSTED BOX OFFICE').cast("Long"))

In [301]:
missing_val_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- TICKETS SOLD: integer (nullable = true)
 |-- TOTAL BOX OFFICE: string (nullable = true)
 |-- TOTAL INFLATION ADJUSTED BOX OFFICE: long (nullable = true)
 |-- AVERAGE TICKET PRICE: string (nullable = true)



In [302]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=["TICKETS SOLD", "TOTAL INFLATION ADJUSTED BOX OFFICE"],
    outputCols=["{}_imputed".format(c) for c in ["TICKETS SOLD", "TOTAL INFLATION ADJUSTED BOX OFFICE"]],
).setStrategy("mean")

In [303]:
# Add imputation cols to df

imputer.fit(missing_val_df).transform(missing_val_df).show()

+----+------------+----------------+-----------------------------------+--------------------+--------------------+-------------------------------------------+
|YEAR|TICKETS SOLD|TOTAL BOX OFFICE|TOTAL INFLATION ADJUSTED BOX OFFICE|AVERAGE TICKET PRICE|TICKETS SOLD_imputed|TOTAL INFLATION ADJUSTED BOX OFFICE_imputed|
+----+------------+----------------+-----------------------------------+--------------------+--------------------+-------------------------------------------+
|2021|   423774881|  $3,881,777,912|                         3881777912|               $9.16|           423774881|                                 3881777912|
|2020|   223638958|  $2,048,534,616|                         2048534616|               $9.16|           223638958|                                 2048534616|
|2019|  1228541629| $11,253,443,955|                        11253444050|               $9.16|          1228541629|                                11253444050|
|2018|  1311536128| $11,948,096,650|          